# Interactive Dashboard

This notebook contains the code for the interactive dashboard to visualize data in a more accessible way.

In [1]:
import os
import ast
import pandas as pd
import plotly as pl
import plotly.express as px
import panel as pn
import panel.widgets as pnw

## Load dataset

NOTE: Be sure to run previous notebooks first before starting here

In [2]:
# Get the directory where the notebook file lives
base_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()

# Construct the full path to the CSV file
data_path = os.path.join(base_dir.replace('/notebooks', ''), "data", "steamdataset.csv")

print("Loading dataset from:", data_path)

Loading dataset from: /Users/hammy/Projects/DataVisProj/data/steamdataset.csv


In [3]:

def load_dataset():
    df = pd.read_csv(data_path, parse_dates=['date', 'release_date'])
    df.dropna(inplace=True)
    
    # Convert string to list if needed, then strip and lowercase, take top 5, and convert to set
    df['genres'] = df['genres'].apply(
        lambda x: set([g.strip().lower() for g in (ast.literal_eval(x) if isinstance(x, str) else x)][:5])
    )
    
    return df

steamdataset = load_dataset()

In [4]:
import itertools

all_genres = list(itertools.chain.from_iterable(steamdataset['genres']))
unique_genres = sorted(set(all_genres))
unique_games = sorted(steamdataset['game_name'].unique())


print("Unique genres found:", len(unique_genres))
print(unique_genres[:20])  # check first 20 genres

Unique genres found: 200
['2d', '3d', '3d fighter', '3d platformer', '4x', 'action', 'action roguelike', 'action rpg', 'action rts', 'action-adventure', 'adventure', 'america', 'arcade', 'atmospheric', 'atv', 'automation', 'automobile sim', 'base building', 'baseball', 'basketball']


In [5]:
rating_map = {
    'Overwhelmingly Positive': 5,
    'Very Positive': 4,
    'Positive': 3,
    'Mixed': 2,
    'Negative': 1,
    'Very Negative': 0
}
steamdataset['rating_num'] = steamdataset['overall_player_rating'].map(rating_map)
steamdataset['days_since_release'] = (pd.Timestamp.today() - steamdataset['release_date']).dt.days

## Helper Functions

In [6]:
def filter_data(genres=None, recommendation=None, date_range=None, selected_games=None, playtime_range=None, rec_ratio_range=None):
    df_filtered = steamdataset

    # Filter recommendation (cheap)
    if recommendation:
        df_filtered = df_filtered[df_filtered['recommendation'].isin(recommendation)]

    # Filter date range (vectorized)
    if date_range:
        start_date, end_date = pd.Timestamp(date_range[0]), pd.Timestamp(date_range[1])
        df_filtered = df_filtered[df_filtered['date'].between(start_date, end_date)]

    # Filter genres (vectorized with sets)
    if genres:
        selected_genres_set = set([g.lower() for g in genres])
        # Use set intersection to check if any selected genre is in the game's genre set
        df_filtered = df_filtered[df_filtered['genres'].apply(lambda x: bool(x & selected_genres_set))]

    # Filter selected games
    if selected_games:
        df_filtered = df_filtered[df_filtered['game_name'].isin(selected_games)]
    
    # Filter playtime range (vectorized)
    if playtime_range:
        min_playtime, max_playtime = playtime_range
        df_filtered = df_filtered[df_filtered['hours_played'].between(min_playtime, max_playtime)]
    
    if rec_ratio_range:
        min_ratio, max_ratio = rec_ratio_range
        df_filtered = df_filtered[df_filtered['rec_ratio'].between(min_ratio, max_ratio)]

    return df_filtered

In [7]:
def create_comparison_graph(genres, recommendation, date_range, selected_games):
    df_filtered = filter_data(genres, recommendation, date_range, selected_games)
    
    if df_filtered.empty:
        fig = px.line(title="No data for selected filters")
    else:
        # If no games selected, pick top 3 by number of reviews
        if not selected_games:
            top_games = df_filtered['game_name'].value_counts().nlargest(3).index.tolist()
        else:
            top_games = selected_games

        df_top = df_filtered[df_filtered['game_name'].isin(top_games)]
        df_grouped = df_top.groupby(['date', 'game_name']).size().reset_index(name='review_count')

        fig = px.line(
            df_grouped, x='date', y='review_count', color='game_name',
            title="Review Trends by Game",
            labels={'review_count':'Number of Reviews', 'date':'Date', 'game_name':'Game'}
        )
    return fig


In [8]:
def create_trend_graph(genres, recommendation, date_range, selected_games):
    df_filtered = filter_data(genres, recommendation, date_range, selected_games)
    
    if df_filtered.empty:
        fig = px.line(title="No data for selected filters")
    else:
        df_grouped = df_filtered.groupby('date').size().reset_index(name='review_count')
        fig = px.line(df_grouped, x='date', y='review_count', 
                      title="Review Trend Over Time",
                      labels={'review_count':'Number of Reviews', 'date':'Date'})
    return fig

## Components

In [9]:

genre_widget = pn.widgets.MultiChoice(name="Genres", options=unique_genres)
review_widget = pn.widgets.MultiChoice(name="Review Type", options=[0, 1], value=[1])
date_widget = pn.widgets.DateRangeSlider(
    name="Date Range", 
    start=steamdataset['date'].min(), 
    end=steamdataset['date'].max(), 
    value=(steamdataset['date'].min(), steamdataset['date'].max())
)
game_widget = pn.widgets.MultiChoice(
    name="Game",
    options=unique_games,
    value=[],
)

playtime_min = pn.widgets.NumberInput(name="Min Playtime", value=0, step=0.1)
playtime_max = pn.widgets.NumberInput(
    name="Max Playtime",
    value=float(steamdataset['hours_played'].max()),  # initial value = max
    start=0,
    end=float(steamdataset['hours_played'].max()),   # constrain range
    step=0.1
)

playtime_widget = pn.Column(playtime_min, playtime_max, sizing_mode='stretch_width')

rec_ratio_min = pn.widgets.FloatInput(name="Min Recommendation Ratio", value=0.0, start=0.0, end=1.0, step=0.1)
rec_ratio_max = pn.widgets.FloatInput(
    name="Max Recommendation Ratio",
    value=1.0,
    start=0.0,
    end=1.0,
    step=0.1
)

rec_ratio_widget = pn.Column(rec_ratio_min, rec_ratio_max, sizing_mode='stretch_width')

numerical_columns = ['hours_played', 'rec_ratio', 'helpful', 'funny', 'recommendation', 'popular', 'days_since_release', 'rating_num']


x_axis = pn.widgets.Select(name='X-axis', options=numerical_columns, value='hours_played')
y_axis = pn.widgets.Select(name='Y-axis', options=numerical_columns, value='recommendation')


In [10]:
overview_panel = pn.Column(
    pn.pane.Markdown("## Overview of Steam Game Reviews"),
    pn.pane.Markdown("This dataset contains approximately 990K reviews from Steam users, covering a wide range of games and genres. The dashboard allows you to explore review trends over time and compare review patterns across different games. On the left sidebar, you can select what type of data/graph you'd like to look at and filter the data by genres, review type (positive/negative), date range, and specific games. Here in the center, you will see the visualizations update based on your selections. Please note that the dataset is quite large, so some graphs may take a moment to load after changing filters."),
    pn.pane.Markdown("---"),
    pn.pane.Markdown("## Dataset Summary"),
    pn.pane.Markdown(f"- Total Reviews: {len(steamdataset)}"),
    pn.pane.Markdown(f"- Date Range: {steamdataset['date'].min().date()} to {steamdataset['date'].max().date()}"),
    pn.pane.Markdown(f"- Unique Games: {steamdataset['game_name'].nunique()}"),
    pn.pane.Markdown(f"- Unique Genres: {len(unique_genres)}"),
    pn.pane.Markdown(f"- Recommendation Types: Positive (1) ({(steamdataset['recommendation'] == 1).sum()}), Negative (0) ({(steamdataset['recommendation'] == 0).sum()})"),
    pn.pane.Markdown("A few other included features are: review text, user playtime at review, game release date, helpful/funny votes, and recommendation ratios."),
    pn.pane.Markdown("---"),
    pn.pane.Markdown("## Team Gamers"),
    pn.pane.Markdown("- Lawrence Cuenco"),
    pn.pane.Markdown("- Nathan Wong"),
    pn.pane.Markdown("- Jake Johnson"),
    pn.pane.Markdown("- Sai Vishnumolakala"),
)

In [11]:
import panel as pn
import plotly.express as px

pn.extension('plotly')

# --- Graph panels (reactive) ---
trend_panel = pn.panel(
    pn.bind(
        create_trend_graph,
        genres=genre_widget,
        recommendation=review_widget,
        date_range=date_widget,
        selected_games=game_widget
    ),
    sizing_mode='stretch_both',
    loading_indicator=True
)

compare_panel = pn.panel(
    pn.bind(
        create_comparison_graph,
        genres=genre_widget,
        recommendation=review_widget,
        date_range=date_widget,
        selected_games=game_widget
    ),
    sizing_mode='stretch_both',
    loading_indicator=True
)

# Tabs for graphs
line_tabs = pn.Tabs(
    ("Overall Trend", trend_panel),
    ("Game Comparison", compare_panel)
)

# Container for the graph panel
graph_panel = pn.Column(line_tabs, sizing_mode='stretch_both')


In [12]:
pn.extension('tabulator')

selected_cols = ['game_name', 'recommendation', 'date', 'hours_played', 'release_date', 'genres', 'helpful', 'funny', 'rec_ratio']

table_widget = pn.widgets.Tabulator(
    steamdataset[selected_cols],  # initial value
    pagination='remote',
    page_size=25,
    sizing_mode='stretch_both',
    selectable=True
)


def update_table(event=None):
    # Apply filtering
    df_filtered = filter_data(
        genres=genre_widget.value,
        recommendation=review_widget.value,
        date_range=date_widget.value,
        selected_games=game_widget.value,
        playtime_range=(playtime_min.value, playtime_max.value),
        rec_ratio_range=(rec_ratio_min.value, rec_ratio_max.value)
    ).copy()
    df_filtered['genres'] = df_filtered['genres'].apply(lambda x: ', '.join(x))
    
    # Update the table without recreating
    table_widget.value = df_filtered[selected_cols]

# Watch all filter widgets
for w in [genre_widget, review_widget, date_widget, game_widget, playtime_min, playtime_max, rec_ratio_min, rec_ratio_max]:
    w.param.watch(update_table, 'value')

table_panel = pn.Column(
    pn.pane.Markdown("## Steam Reviews Table"),
    table_widget,
    sizing_mode='stretch_both'
)

In [13]:
selected_genres = ['action', 'adventure', 'rpg', 'indie', 'strategy', 'sports', 'simulation', 'casual', 'fps'] # These are like the primary genres

pn.extension('plotly')

df_exploded = steamdataset.explode('genres')
df_filtered = df_exploded[df_exploded['genres'].str.lower().isin(selected_genres)]
df_counts = df_filtered.groupby(['date', 'genres']).size().reset_index(name='review_count')

def small_multiples_graph(date_range):
    start_date, end_date = pd.Timestamp(date_range[0]), pd.Timestamp(date_range[1])
    df_counts_filtered = df_counts[df_counts['date'].between(start_date, end_date)]
    fig = px.line(
        df_counts_filtered, x='date', y='review_count', color='genres',
        facet_col='genres', facet_col_wrap=3,
        labels={'review_count':'Number of Reviews', 'date':'Date', 'genres':'Genre'}
    )
    fig.update_layout(showlegend=False)
    return fig


small_multiples_panel = pn.Column(
    pn.pane.Markdown("## Review Trends by Genre (Small Multiples)"),
    pn.panel(
        pn.bind(
            small_multiples_graph,
            date_range=date_widget
        ),
        sizing_mode='stretch_both',
        loading_indicator=True
    )
)

In [14]:
def scatter_plot(x, y):
    fig = px.scatter(steamdataset.sample(n=10000), x=x, y=y, hover_data=numerical_columns, title=f"{y} vs {x}")
    return fig

# Panel layout
scatter_panel = pn.Column(
    pn.pane.Markdown("## Scatter Plot Explorer"),
    pn.Row(x_axis, y_axis),
    pn.panel(
        pn.bind(
            scatter_plot,
            x=x_axis,
            y=y_axis
        ),
        sizing_mode='stretch_both',
        loading_indicator=True
    )
)

# App

In [15]:
pn.extension('plotly')

sidebar_accordion = pn.Accordion(
    ("Overview", pn.Column()),
    ("Review Count", pn.Column(game_widget, genre_widget, review_widget, date_widget)),
    ("Table View", pn.Column(game_widget, genre_widget, review_widget, date_widget, playtime_widget, rec_ratio_widget)),
    ("Main Genres", pn.Column(date_widget)),
    ("Compare Plots", pn.Column(x_axis, y_axis)),
    active=[0],
    toggle=True,
)

overview_panel.visible = True
graph_panel.visible = False
table_panel.visible = False
small_multiples_panel.visible = False
scatter_panel.visible = False
main_panel = pn.Column(overview_panel, graph_panel, table_panel, small_multiples_panel, scatter_panel, sizing_mode='stretch_both')

# Function to switch accordion
def switch_accordion(event):
    # event.new can be empty if user closes last open section
    if event.new:
        last_open = event.new[-1]
        # Force only that one to be active
        sidebar_accordion.active = [last_open]
        # Show/hide main panels
        overview_panel.visible = (last_open == 0)
        graph_panel.visible = (last_open == 1)
        table_panel.visible = (last_open == 2)
        small_multiples_panel.visible = (last_open == 3)
        scatter_panel.visible = (last_open == 4)
    else:
        # If user closes everything, fallback to Overview
        sidebar_accordion.active = [0]
        overview_panel.visible = True
        graph_panel.visible = table_panel.visible = small_multiples_panel.visible = scatter_panel.visible = False

# Watch changes
sidebar_accordion.param.watch(switch_accordion, 'active')

template = pn.template.MaterialTemplate(
    title="Steam Game Reviews Dashboard",
    sidebar=[sidebar_accordion],
    main=[main_panel]
)

template.show()


Launching server at http://localhost:58344


In [ ]:
template.servable()

MaterialTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-15234459040] Column(sizing_mode='stretch_both')
        [0] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] Markdown(str)
            [3] Markdown(str)
            [4] Markdown(str)
            [5] Markdown(str)
            [6] Markdown(str)
            [7] Markdown(str)
            [8] Markdown(str)
            [9] Markdown(str)
            [10] Markdown(str)
            [11] Markdown(str)
            [12] Markdown(str)
            [13] Markdown(str)
            [14] Markdown(str)
            [15] Markdown(str)
        [1] Column(sizing_mode='stretch_both', visible=False)
            [0] Tabs
                [0] ParamFunction(function, _pane=Plotly, defer_load=False, loading_indicator=True, sizing_mode='stretch_both')
                [1] ParamFunction(function, _pane=Plotly, defer_load=False, loading_indicator=True, sizing_mode='stretch_both')
        [2] Column(sizing_mode='stretch_both', visible=False)
            [0] Markdown(str)
            [1] Tabulator(page_size=25, pagination='remote', sizing_mode='stretch_both', value=                          ...)
        [3] Column(visible=False)
            [0] Markdown(str)
            [1] ParamFunction(function, _pane=Plotly, defer_load=False, loading_indicator=True, sizing_mode='stretch_both')
        [4] Column(visible=False)
            [0] Markdown(str)
            [1] Row
                [0] Select(name='X-axis', options=['hours_played', ...], value='hours_played')
                [1] Select(name='Y-axis', options=['hours_played', ...], value='recommendation')
            [2] ParamFunction(function, _pane=Plotly, defer_load=False, loading_indicator=True, sizing_mode='stretch_both')
    [nav-15234450208] Accordion(active=[0], toggle=True)
        [0] Column()
        [1] Column
            [0] MultiChoice(name='Game', options=['69 Balls', '7 Days to Di...])
            [1] MultiChoice(name='Genres', options=['2d', '3d', '3d fighter',...])
            [2] MultiChoice(name='Review Type', options=[0, 1], value=[1])
            [3] DateRangeSlider(end=Timestamp('2024-09-23 0..., name='Date Range', start=Timestamp('2010-10-16 0..., value=(Timestamp('2010-10-16 00:..., value_end=Timestamp('2024-09-23 0..., value_start=Timestamp('2010-10-16 0...)
        [2] Column
            [0] MultiChoice(name='Game', options=['69 Balls', '7 Days to Di...])
            [1] MultiChoice(name='Genres', options=['2d', '3d', '3d fighter',...])
            [2] MultiChoice(name='Review Type', options=[0, 1], value=[1])
            [3] DateRangeSlider(end=Timestamp('2024-09-23 0..., name='Date Range', start=Timestamp('2010-10-16 0..., value=(Timestamp('2010-10-16 00:..., value_end=Timestamp('2024-09-23 0..., value_start=Timestamp('2010-10-16 0...)
            [4] Column(sizing_mode='stretch_width')
                [0] FloatInput(name='Min Playtime')
                [1] FloatInput(end=107075.1, name='Max Playtime', start=0, value=107075.1)
            [5] Column(sizing_mode='stretch_width')
                [0] FloatInput(end=1.0, name='Min Recommendation R..., start=0.0)
                [1] FloatInput(end=1.0, name='Max Recommendation R..., start=0.0, value=1.0)
        [3] Column
            [0] DateRangeSlider(end=Timestamp('2024-09-23 0..., name='Date Range', start=Timestamp('2010-10-16 0..., value=(Timestamp('2010-10-16 00:..., value_end=Timestamp('2024-09-23 0..., value_start=Timestamp('2010-10-16 0...)
        [4] Column
            [0] Select(name='X-axis', options=['hours_played', ...], value='hours_played')
            [1] Select(name='Y-axis', options=['hours_played', ...], value='recommendation')